<a href="https://www.kaggle.com/code/emineyetm/kural-tabanl-s-n-fland-rma?scriptVersionId=112453488" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/persona/persona.csv


**İş Problemi**

Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak seviye tabanlı (level based) yeni müşteri tanımları (persona) oluşturmak ve bu yeni müşteri tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.

Örneğin: Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek kullanıcının ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

**Veri Seti Hikayesi

Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu ürünleri satın alan kullanıcıların bazı demografik bilgilerini barındırmaktadır. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla alışveriş yapmış olabilir.

* Price: Müşterinin harcama tutarı
* Source: Müşterinin bağlandığı cihaz türü
* Sex: Müşterinin cinsiyeti
* Country: Müşterinin ülkesi
* Age: Müşterinin yaşı

################# Uygulama Öncesi #####################

   PRICE   SOURCE   SEX COUNTRY  AGE
* 0     39  android  male     bra   17
* 1     39  android  male     bra   17
* 2     49  android  male     bra   17
* 3     29  android  male     tur   17
* 4     49  android  male     tur   17

################# Uygulama Sonrası #####################

      customers_level_based        PRICE SEGMENT
* 0  BRA_ANDROID_FEMALE_0_18  1139.800000        A
* 1  BRA_ANDROID_FEMALE_19_23  1070.600000       A
* 2  BRA_ANDROID_FEMALE_24_30   508.142857       A
* 3  BRA_ANDROID_FEMALE_31_40   233.166667       C
* 4  BRA_ANDROID_FEMALE_41_66   236.666667       C

# **GÖREV 1:** 

Aşağıdaki soruları yanıtlayınız.

*Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.*

In [2]:
# kütüphane import edilir.
import pandas as pd

In [3]:
df = pd.read_csv("../input/persona/persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [4]:

# Soru 2: Kaç unique SOURCE vardır? Frekansları nedir?

df["SOURCE"].nunique()
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [5]:
# Soru 3: Kaç unique PRICE vardır?
df["PRICE"].nunique()

6

In [6]:
# Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş?

df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [7]:
# Soru 5: Hangi ülkeden kaçar tane satış olmuş?

df["COUNTRY"].value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [8]:
# Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış?

df.groupby("COUNTRY")["PRICE"].sum()

#groupby-agg fonksiyonu ile çıktı daha güzel görünüyor.
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [9]:
# Soru 7: SOURCE türlerine göre göre satış sayıları nedir?

df.groupby("SOURCE")["PRICE"].count()

SOURCE
android    2974
ios        2026
Name: PRICE, dtype: int64

In [10]:
#groupby-agg fonksiyonu ile çıktı daha güzel görünüyor.
df.groupby("SOURCE").agg({"PRICE": "count"})

,PRICE
SOURCE,
android,2974
ios,2026


In [11]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?

df.groupby("COUNTRY")["PRICE"].mean()

COUNTRY
bra    34.327540
can    33.608696
deu    34.032967
fra    33.587459
tur    34.787140
usa    34.007264
Name: PRICE, dtype: float64

In [12]:
#groupby-agg fonksiyonu ile çıktı daha güzel görünüyor.
df.groupby('COUNTRY').agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [13]:
# Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?

df.groupby("SOURCE")["PRICE"].mean()

df.groupby('SOURCE').agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [14]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?

df.groupby(["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

# **GÖREV 2:** 

COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?


In [15]:
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

# **GÖREV 3:** 
Çıktıyı PRICE'a göre sıralayınız.

* Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
* Çıktıyı agg_df olarak kaydediniz.
* !!sort values'daki by default olarka yer aldığı için yazmasakta olur.

In [16]:
agg_df = df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values(by=["PRICE"], ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


# **GÖREV 4:** 

Indekste yer alan isimleri değişken ismine çeviriniz.

* Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
* Bu isimleri değişken isimlerine çeviriniz.
* İpucu: reset_index()
* agg_df.reset_index(inplace=True)

In [17]:
agg_df.reset_index(inplace=True)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


# **GÖREV 5:** 

AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.

* Age sayısal değişkenini kategorik değişkene çeviriniz.
* Aralıkları ikna edici olacağını düşündüğünüz şekilde oluşturunuz.
* Örneğin: '0_18', '19_23', '24_30', '31_40', '41_70'

In [18]:
# pd.cut ile AGE değişkeninin nerelerden bölüneceği belirlenir.#bins: Segmentasyon için bin sınırlarını belirtir.
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=['0_18', '19_23', '24_30', '31_40', '41_70'])
agg_df["AGE_CAT"].head()

0    41_70
1    31_40
2    24_30
3    31_40
4    31_40
Name: AGE_CAT, dtype: category
Categories (5, object): ['0_18' < '19_23' < '24_30' < '31_40' < '41_70']

In [19]:
agg_df["AGE_CAT"].value_counts()

24_30    78
0_18     77
19_23    71
31_40    65
41_70    57
Name: AGE_CAT, dtype: int64

# **GÖREV 6:** 

Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.

* customers_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
* Dikkat!
* list comp ile customers_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir.
* Örneğin birden fazla şu ifadeden olabilir: USA_ANDROID_MALE_0_18
* Bunları groupby'a alıp price ortalamalarını almak gerekmektedir.

In [20]:
agg_df["customers_level_based"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,41_70,BRA_ANDROID_MALE_41_70
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [21]:
agg_df = agg_df[["customers_level_based", "PRICE"]]
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41_70,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0


In [22]:
agg_df["customers_level_based"].value_counts()

BRA_ANDROID_MALE_24_30      7
USA_ANDROID_MALE_41_70      7
USA_IOS_FEMALE_24_30        7
BRA_ANDROID_FEMALE_24_30    7
USA_ANDROID_MALE_24_30      7
                           ..
TUR_ANDROID_MALE_41_70      1
CAN_ANDROID_MALE_19_23      1
TUR_IOS_MALE_31_40          1
TUR_IOS_MALE_24_30          1
CAN_ANDROID_FEMALE_24_30    1
Name: customers_level_based, Length: 109, dtype: int64

In [23]:
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})
agg_df.head()

,PRICE
customers_level_based,
BRA_ANDROID_FEMALE_0_18,35.645303
BRA_ANDROID_FEMALE_19_23,34.077340
BRA_ANDROID_FEMALE_24_30,33.863946
BRA_ANDROID_FEMALE_31_40,34.898326
BRA_ANDROID_FEMALE_41_70,36.737179


# **GÖREV 7:** 

Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.

* PRICE'a göre segmentlere ayırınız,
* segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz,
* segmentleri betimleyiniz.

In [24]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])

In [25]:
agg_df.head(30)

,PRICE,SEGMENT
customers_level_based,,
BRA_ANDROID_FEMALE_0_18,35.645303,B
BRA_ANDROID_FEMALE_19_23,34.077340,C
BRA_ANDROID_FEMALE_24_30,33.863946,C
BRA_ANDROID_FEMALE_31_40,34.898326,B
BRA_ANDROID_FEMALE_41_70,36.737179,A
BRA_ANDROID_MALE_0_18,34.805861,B
BRA_ANDROID_MALE_19_23,31.673243,D
BRA_ANDROID_MALE_24_30,33.413919,C
BRA_ANDROID_MALE_31_40,34.327381,B


In [26]:
agg_df.groupby("SEGMENT").agg({"PRICE": ["mean", "max", "sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        29.206780  32.333333   817.789833
C        33.509674  34.077340   904.761209
B        34.999645  36.000000   944.990411
A        38.691234  45.428571  1044.663328